# 并发编程
---

### 信号控制线程启动

假如程序中的其他线程需要通过判断某个线程的状态来确定自己下一步的操作  
可以使用 `threading` 库中的 `Event` 对象来作为全局信号标志，调用 `wait` 方法使线程阻塞，当调用 `set` 方法后，所有阻塞线程会收到信号，才会继续执行

In [1]:
from threading import Thread, Event
import time

def countdown(n, started_evt):
    print('countdown starting')
    started_evt.set()
    while n > 0:
        time.sleep(3)
        print('T-minus', n)
        n -= 1

started_evt = Event()

print('Launching countdown')
t = Thread(target=countdown, args=(5,started_evt))
t.start()

# 这里会等待 Event 对象被执行 set
started_evt.wait()
print('countdown is running')
t.join()

Launching countdown
countdown starting
countdown is running
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


---
`Event` 对象最好不要复用，当 `set` 被调用后，应该重新创建新对象，因为虽然可以使用 `clear` 方法重置，但无法保证重置时是在线程再次等待之前。  
如果需要复用，可以通过 `Condition` 对象来代替

In [2]:
import threading
import time


class PeriodicTimer:
    """ 周期定时器 """

    def __init__(self, interval):
        self._interval = interval
        self._flag = 0
        self._cv = threading.Condition()

    def start(self):
        t = threading.Thread(target=self.run)
        t.daemon = True
        t.start()

    def run(self):
        """
        启动定时器，并每隔一段时间发出通知
        """
        while True:
            time.sleep(self._interval)
            with self._cv:
                # 异或，相异为1，相同为0，达到1与0来回切换的效果
                self._flag ^= 1
                self._cv.notify_all()

    def wait_for_tick(self):
        """
        等待下次通知
        """
        with self._cv:
            # 当 flag 发生切换，则停止阻塞
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait()


ptimer = PeriodicTimer(5)
ptimer.start()


def countdown(nticks):
    while nticks > 0:
        ptimer.wait_for_tick()
        print('T-minus %d' % nticks)
        nticks -= 1

def countup(last):
    n = 0
    while n < last:
        ptimer.wait_for_tick()
        print('Counting %d' % n)
        n += 1

t1 = threading.Thread(target=countdown, args=(5,))
t2 = threading.Thread(target=countup, args=(3,))
t1.start()
t2.start()
t1.join()
t2.join()

T-minus 5
Counting 0
T-minus 4Counting 1

Counting 2
T-minus 3
T-minus 2
T-minus 1


---
`Semaphore` 对象和 `BoundedSemaphore` 对象可以通知单一线程（动态控制线程数）  
当调用 `acquire` 方法时，线程尝试获取一个信号，如果没有接收到一直阻塞  
当调用 `release` 方法会释放一个信号，这时只有最早阻塞的线程会收到信号并解除阻塞  
这两个对象的区别是，`Semaphore` 对象如果调用 `release` 方法时没有线程在 `acquire` 状态，会将信号存储用于下一个 `acquire` ，而 `BoundedSemaphore` 此时会报错

In [3]:
def worker(n, sema):
    sema.acquire()
    print('Working', n)


# 参数是初始的信号数
sema = threading.Semaphore(0)
nworkers = 10
for n in range(nworkers):
    t = threading.Thread(target=worker, args=(n, sema,))
    t.start()

In [4]:
sema.release()

Working 0


In [5]:
sema.release()

Working 1


In [6]:
sema.release()

Working 2


---

### 线程间通信

In [7]:
import queue
from threading import Thread, Event

def producer(letter_list, out_q):
    letters = letter_list[:]
    while True:
        if not letters:
            break
        letter = letters.pop()
        evt = Event()
        out_q.put((letter, evt))
        # 等待消费者处理完
        evt.wait()
        # 随后可以有一些其他操作再循环下次
        print(' ', end='')

def consumer(in_q):
    while True:
        try:
            letter, evt = in_q.get(timeout=3)
        except queue.Empty:
            break
        print(letter.upper(), end='')
        # 通知已经完成消费
        evt.set()

q = queue.Queue()
letters = ['a', 'b', 'c', 'd']
t1 = Thread(target=consumer, args=(q,))
t2 = Thread(target=producer, args=(letters, q))
t1.start()
t2.start()

D C B A 

### 多线程加锁

In [8]:
import threading


class SharedCounter:

    def __init__(self, initial_value = 0):
        self._value = initial_value
        self._value_lock = threading.Lock()

    def incr(self,delta=1):
        with self._value_lock:
             self._value += delta

    def decr(self,delta=1):
        with self._value_lock:
             self._value -= delta

`Lock` 对象的锁机制，设定为同一线程只允许获取一个锁，如果需要多次获取，可以使用 `RLock` 对象

In [9]:
import threading


class SharedCounter:
    # 类级锁，这样无论该类有多少个实例都只用一个锁
    _lock = threading.RLock()

    def __init__(self, initial_value = 0):
        self._value = initial_value

    def incr(self, delta=1):
        with SharedCounter._lock:
             self._value += delta

    def decr(self, delta=1):
        # 多重获取锁
        with SharedCounter._lock:
             self.incr(-delta)

---

### 防止死锁的加锁机制

在多线程程序中，死锁问题很大一部分是由于线程同时获取多个锁造成的。  
举个例子：一个线程获取了第一个锁，然后在获取第二个锁的时候发生阻塞，那么这个线程就可能阻塞其他线程的执行，从而导致整个程序假死。  
解决死锁问题的一种方案是为程序中的每一个锁分配一个唯一的ID，然后只允许按照升序规则来使用多个锁，这个规则使用上下文管理器是非常容易实现的。

In [10]:
import threading
from contextlib import contextmanager

_local = threading.local()

@contextmanager
def acquire(*locks):
    locks = sorted(locks, key=lambda x: id(x))

    acquired = getattr(_local, 'acquired', [])
    if acquired and max(id(lock) for lock in acquired) >= id(locks[0]):
        raise RuntimeError('Lock Order Violation')

    acquired.extend(locks)
    _local.acquired = acquired

    try:
        for lock in locks:
            lock.acquire()
        yield
    finally:
        for lock in reversed(locks):
            lock.release()
        del acquired[-len(locks):]

In [11]:
import threading

x_lock = threading.Lock()
y_lock = threading.Lock()

def thread_1():
    for _ in range(10):
        with acquire(x_lock, y_lock):
            print('Thread-1')

def thread_2():
    for _ in range(10):
        with acquire(y_lock, x_lock):
            print('Thread-2')

t1 = threading.Thread(target=thread_1)
t1.daemon = True

t2 = threading.Thread(target=thread_2)
t2.daemon = True

t1.start()
t2.start()

Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2


---

### 保存线程的状态信息

假设需要实现一个支持多线程的惰性连接器，使用 `thread.local()` 可以让 LazyConnection 类支持一个线程一个连接，而不是所有的进程都共用一个连接。  
其原理是，每个 `threading.local()` 实例为每个线程维护着一个单独的实例字典。所有普通实例操作比如获取、修改和删除值仅仅操作这个字典。 每个线程使用一个独立的字典就可以保证数据的隔离了。

In [12]:
from socket import socket, AF_INET, SOCK_STREAM
import threading


class LazyConnection:

    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.local = threading.local()

    def __enter__(self):
        """ 每个线程会创建自己专属的 socket 连接，线程之间互不影响 """
        if hasattr(self.local, 'sock'):
            raise RuntimeError('Already connected')
        self.local.sock = socket(self.family, self.type)
        self.local.sock.connect(self.address)
        return self.local.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.local.sock.close()
        del self.local.sock

In [13]:
from functools import partial

def test(conn):
    with conn as s:
        s.send(b'GET /index.html HTTP/1.0\r\n')
        s.send(b'Host: www.python.org\r\n')
        s.send(b'\r\n')
        resp = b''.join(iter(partial(s.recv, 8192), b''))

    print('Got {} bytes'.format(len(resp)))

In [14]:
conn = LazyConnection(('www.python.org', 80))

t1 = threading.Thread(target=test, args=(conn,))
t2 = threading.Thread(target=test, args=(conn,))
t1.start()
t2.start()
t1.join()
t2.join()

Got 392 bytes
Got 392 bytes


---

### 创建一个线程池

In [15]:
from socket import AF_INET, SOCK_STREAM, socket
from concurrent.futures import ThreadPoolExecutor
from queue import Queue

def echo_client(q):
    sock, client_addr = q.get()
    print('Got connection from', client_addr)
    while True:
        msg = sock.recv(65536)
        if not msg:
            break
        sock.sendall(msg)
    print('Client closed connection')
    sock.close()

def echo_server(addr, nworkers):
    q = Queue()
    for n in range(nworkers):
        t = Thread(target=echo_client, args=(q,))
        t.daemon = True
        t.start()

    sock = socket(AF_INET, SOCK_STREAM)
    sock.bind(addr)
    sock.listen(5)
    while True:
        client_sock, client_addr = sock.accept()
        q.put((client_sock, client_addr))

In [16]:
from threading import Thread

# 应注意限制最大线程数
t = Thread(target=echo_server, 
           args=(('', 15000), 128))
t.daemon = True
t.start()

In [17]:
from concurrent.futures import ThreadPoolExecutor
import urllib.request

def fetch_url(url):
    u = urllib.request.urlopen(url)
    data = u.read()
    return data

pool = ThreadPoolExecutor(10)

a = pool.submit(fetch_url, 'http://www.python.org')
b = pool.submit(fetch_url, 'http://www.pypy.org')

# result 方法会阻塞进程直到对应的函数执行完成并返回一个结果
x = a.result()
y = b.result()

In [18]:
x[:1000]

b'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->\n<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->\n<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->\n\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">\n\n    <meta name="application-name" content="Python.org">\n    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">\n    <meta name="apple-mobile-web-app-title" content="Python.org">\n    <meta name="apple-mobile-web-app-capable" content="yes">\n    <meta name="apple-mobile-web-app-status-bar-style" content="black">\n\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="Ha

In [19]:
y[:1000]

b'<!DOCTYPE html>\n<html>\n<head>\n\t<title>PyPy - Welcome to PyPy</title>\n\t<meta http-equiv="content-language" content="en" />\n\t<meta http-equiv="content-type" content="text/html; charset=utf-8" />\n\t<meta name="author" content="PyPy Team" />\n\t<meta name="description" content="PyPy" />\n\t<meta name="copyright" content="MIT" />\n\t<meta name="document-rating" content="general" />\n\t<link rel="stylesheet" type="text/css" media="screen" title="default" href="css/site.css" />\n\t<link rel="alternate" type="application/rss+xml" title="RSS Feed for PyPy" href="http://feeds.feedburner.com/PyPyStatusBlog" />\n  <link rel="stylesheet" type="text/css" href="css/jquery-ui-1.8.14.custom.css" />\n\t<script type="text/javascript" src="https://use.typekit.com/hdt8sni.js"></script>\n\t<script type="text/javascript">try{Typekit.load();}catch(e){}</script>\n\t<script type="text/javascript" src="https://ajax.googleapis.com/ajax/libs/jquery/1.4.2/jquery.min.js"></script>\n  <script type="text/ja

---

### 简单的并行编程

In [20]:
def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

方法一，使用 `pool.map` 返回生成器对象

In [21]:
from concurrent.futures import ProcessPoolExecutor

# 可以传入整型限制进程数，默认是系统可用的CPU核心数
with ProcessPoolExecutor() as pool:
    results = pool.map(fib, [30, 30, 30, 30])
    print(list(results))

[1346269, 1346269, 1346269, 1346269]


方法二，使用 `pool.submit` 返回 `Future` 实例，然后调用 `result` 方法进行阻塞并等待结果返回

In [22]:
with ProcessPoolExecutor() as pool:
    future_result = pool.submit(fib, 30)
    r = future_result.result()
    print(r)

1346269


方法三，同样使用 `pool.submit` ，调用 `add_done_callback` 方法可以进行非阻塞的回调

In [23]:
def when_done(r):
    print(r.result())

with ProcessPoolExecutor() as pool:
     future_result = pool.submit(fib, 30)
     future_result.add_done_callback(when_done)

1346269


---

### 定义 Actor 模式任务

In [24]:
from queue import Queue
from threading import Thread, Event


class ActorExit(Exception):
    pass


class Actor:

    def __init__(self):
        self._mailbox = Queue()

    def send(self, msg):
        self._mailbox.put(msg)

    def recv(self):
        msg = self._mailbox.get()
        if msg is ActorExit:
            raise ActorExit()
        return msg

    def close(self):
        self.send(ActorExit)

    def start(self):
        self._terminated = Event()
        t = Thread(target=self._bootstrap)
        t.daemon = True
        t.start()

    def _bootstrap(self):
        try:
            self.run()
        except ActorExit:
            pass
        finally:
            self._terminated.set()

    def join(self):
        self._terminated.wait()

    def run(self):
        while True:
            msg = self.recv()


class PrintActor(Actor):

    def run(self):
        while True:
            msg = self.recv()
            print('Got:', msg)


p = PrintActor()
p.start()
p.send('Hello')
p.send('World')
p.close()
p.join()

Got: Hello
Got: World


---

### 实现消息发布/订阅模型

In [25]:
from contextlib import contextmanager
from collections import defaultdict


class Exchange:

    def __init__(self):
        self._subscribers = set()

    def attach(self, task):
        self._subscribers.add(task)

    def detach(self, task):
        self._subscribers.remove(task)

    @contextmanager
    def subscribe(self, *tasks):
        for task in tasks:
            self.attach(task)
        try:
            yield
        finally:
            for task in tasks:
                self.detach(task)

    def send(self, msg):
        for subscriber in self._subscribers:
            subscriber.send(msg)


_exchanges = defaultdict(Exchange)


def get_exchange(name):
    return _exchanges[name]

In [26]:
class Task:

    def __init__(self, name):
        self.name = name

    def send(self, msg):
        print(self.name, 'send:', msg)


task_a = Task('task_a')
task_b = Task('task_b')

In [27]:
exc = get_exchange('name')

In [28]:
with exc.subscribe(task_a, task_b):
     exc.send('msg1')
     exc.send('msg2')

task_a send: msg1
task_b send: msg1
task_a send: msg2
task_b send: msg2


---

### 使用生成器代替线程

使用生成器实现并发，实际就是利用 `yield` 语句让生成器挂起，然后切换任务执行。

In [29]:
def countdown(n):
    while n > 0:
        print('T-minus', n)
        yield
        n -= 1
    print('Blastoff!')

def countup(n):
    x = 0
    while x < n:
        print('Counting up', x)
        yield
        x += 1

In [30]:
from collections import deque


class TaskScheduler:

    def __init__(self):
        self._task_queue = deque()

    def new_task(self, task):
        self._task_queue.append(task)

    def run(self):
        while self._task_queue:
            task = self._task_queue.popleft()
            try:
                next(task)
                self._task_queue.append(task)
            except StopIteration:
                pass


sched = TaskScheduler()
sched.new_task(countdown(10))
sched.new_task(countdown(5))
sched.new_task(countup(15))
sched.run()

T-minus 10
T-minus 5
Counting up 0
T-minus 9
T-minus 4
Counting up 1
T-minus 8
T-minus 3
Counting up 2
T-minus 7
T-minus 2
Counting up 3
T-minus 6
T-minus 1
Counting up 4
T-minus 5
Blastoff!
Counting up 5
T-minus 4
Counting up 6
T-minus 3
Counting up 7
T-minus 2
Counting up 8
T-minus 1
Counting up 9
Blastoff!
Counting up 10
Counting up 11
Counting up 12
Counting up 13
Counting up 14


---

### 多个线程队列轮询

对于轮询问题，可以创建一对 Socket，然后在其中一个 Socket 上面编写代码来标识存在的数据， 另外一个 Socket 被传给 `select` 函数 或一个类似的轮询获得数据的函数。

In [31]:
import queue
import socket
import os


class PollableQueue(queue.Queue):

    def __init__(self):
        super().__init__()
        # 创建一对 Socket
        if os.name == 'posix':
            self._putsocket, self._getsocket = socket.socketpair()
        else:
            server = socket.socket(
                socket.AF_INET, socket.SOCK_STREAM)
            server.bind(('127.0.0.1', 0))
            server.listen(1)
            self._putsocket = socket.socket(
                socket.AF_INET, socket.SOCK_STREAM)
            self._putsocket.connect(server.getsockname())
            self._getsocket, _ = server.accept()
            server.close()

    def fileno(self):
        return self._getsocket.fileno()

    def put(self, item):
        super().put(item)
        self._putsocket.send(b'x')

    def get(self):
        self._getsocket.recv(1)
        return super().get()

In [32]:
import select
import threading

def consumer(queues):
    while True:
        # 使用 select 来轮询队列的 fileno 方法
        can_read, _, _ = select.select(queues,[],[])
        for r in can_read:
            item = r.get()
            print('Got:', item)

q1 = PollableQueue()
q2 = PollableQueue()
q3 = PollableQueue()

t = threading.Thread(target=consumer, args=([q1,q2,q3],))
t.daemon = True
t.start()

q1.put(1)
q2.put(10)
q3.put('hello')
q2.put(15)

Got: 1
Got: 10
Got: hello
Got: 15
